In [3]:
from generic_qaoa.vqf_helper import create_clauses
from generic_qaoa.clause import MathematicalClause
from generic_qaoa.utils import get_pq_from_selected, plot_histogram
from generic_qaoa import GenericQaoa


In [4]:
m = 56153
p_dict, q_dict, z_dict, clauses = create_clauses(m,true_p_int=233, true_q_int=241, verbose=True)
free_symbols = set().union(*[clause.free_symbols for clause in clauses])
qubit_index_to_symbol = {i: sym for i, sym in enumerate(free_symbols)}
symbol_to_qubit_index = {sym: i for i, sym in qubit_index_to_symbol.items()}
final_clauses = [MathematicalClause((clause * clause).expand(), symbol_to_qubit_index)for clause in clauses if clause != 0]

Preprocessing iteration: 0
Current clause 1 : p_1 + q_1 - 2*z_1_2
Z rule 2 applied: {p_1: q_1, z_1_2: q_1}
Current clause 2 : p_2 + 2*q_1 + q_2 - 2*z_2_3 - 4*z_2_4
Z rule 2 applied: {p_2: q_2}
Current clause 3 : p_3 + 2*q_1*q_2 + q_3 + z_2_3 - 2*z_3_4 - 4*z_3_5 - 1
Current clause 4 : p_3*q_1 + p_4 + q_1*q_3 + q_2 + q_4 + z_2_4 + z_3_4 - 2*z_4_5 - 4*z_4_6 - 1
Current clause 5 : p_3*q_2 + p_4*q_1 + p_5 + q_1*q_4 + q_2*q_3 + q_5 + z_3_5 + z_4_5 - 2*z_5_6 - 4*z_5_7 - 8*z_5_8
Current clause 6 : p_3*q_3 + p_4*q_2 + p_5*q_1 + p_6 + q_1*q_5 + q_2*q_4 + q_6 + z_4_6 + z_5_6 - 2*z_6_7 - 4*z_6_8 - 8*z_6_9 - 1
Current clause 7 : p_3*q_4 + p_4*q_3 + p_5*q_2 + p_6*q_1 + q_1*q_6 + q_2*q_5 + z_5_7 + z_6_7 - 8*z_7_10 - 2*z_7_8 - 4*z_7_9 + 2
Current clause 8 : p_3*q_5 + p_4*q_4 + p_5*q_3 + p_6*q_2 + 2*q_1 + q_2*q_6 + z_5_8 + z_6_8 + z_7_8 - 4*z_8_10 - 8*z_8_11 - 2*z_8_9 - 1
Current clause 9 : p_3*q_6 + p_4*q_5 + p_5*q_4 + p_6*q_3 + 2*q_2 + z_6_9 + z_7_9 + z_8_9 - 2*z_9_10 - 4*z_9_11 - 8*z_9_12 - 1
Curren

In [ ]:
vqf = GenericQaoa(_p=3,
                  _clauses=final_clauses,
                  _qbits=range(len(free_symbols)),
                  _grid_size=8,
                  simulate=True)

vqf.run()
plot_histogram(vqf.last_result.counts_histogram)
p, p_dict, q, q_dict = get_pq_from_selected(p_dict, q_dict, vqf.last_result.selected, symbol_to_qubit_index)
print("p,q=", p, q)

/Users/snir/HW/generic_qaoa/.venv/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/snir/HW/generic_qaoa/.venv/lib/python3.8/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


running circuit on aer_simulator


In [ ]:
if p * q != m:
    print("Trying to fix with bit-flip.")
    for i in range(len(p_dict)):
        for j in range(len(q_dict)):
            new_p: int
            if p_dict[i] == 1:
                new_p = p - 2 ** i
            else:
                new_p = p + 2 ** i
            new_q: int
            if q_dict[j] == 1:
                new_q = q - 2 ** j
            else:
                new_q = q + 2 ** j
            if new_q == m or new_p == m:
                break
            if new_p * q == m:
                p = new_p
            elif p * new_q == m:
                q = new_q
            if new_p * new_q == m:
                p = new_p
                q = new_q
            if p * q == m:
                break
        if p * q == m:
            break
print(p,q)